# Esercizi OCaml - 03 - Tipi Algebrici

## 1. Implementazione pila

**Esercizio 3.1.** Definire due funzioni, `push` e `pop` che, applicate ad una lista di elementi di qualunque tipo, implementino le operazioni tipiche di una pila. In particolare, la funzione `push` prende la lista e un elemento e restituisce una nuova lista con il nuovo elemento aggiunto in testa. La funzione `pop` prende la lista e restituisce una coppia `(elem,lis')` dove `elem` è il primo elemento della lista passata, e `lis'` è la nuova lista, senza il primo elemento. Qualora `pop` sia applicata alla lista vuota. Per gestire il caso in cui si applichi `pop` alla lista vuota, utilizzare un **tipo opzione** per `elem`.

In [18]:
let push pila x = x::pila;;
let pop pila = 
  match pila with 
  | [] -> (None,[])
  | x::pila' -> (Some x, pila');;

val push : 'a list -> 'a -> 'a list = <fun>


val pop : 'a list -> 'a option * 'a list = <fun>


**Esercizio 3.2.** Usando la funzione `pop` definita nell'esercizio precedente, definire una funzione ricorsiva `somma_pila` che, data una lista di interi, calcola la sommma di tutti gli elementi in essa contenuti. **ATTENZIONE:** non usare altri modi per accedere alla lista se non la funzione `pop` (es. non confrontarla con `[]` o decomporla tramite pattern matching).



In [20]:
let rec somma_pila pila =
  match pop pila with
  | (Some v , pila') -> v + somma_pila pila'
  | (None , _) -> 0;;
  
somma_pila [2;4;6];;

val somma_pila : int list -> int = <fun>


- : int = 12


## 2. Implementazione di alberi di decisione

Un albero di decisione è un albero che, dato un insieme di dati in input, consente di determinare un output (la decisione) in funzione di una serie di test successivi effettuati sull'input. Vedere [queste slides](https://www.math.unipd.it/~sperduti/SI08/alberi_decisione.pdf) (solo le prime) per una spiegazione con un esempio.

L'idea è che ogni nodo dell'albero esprime un test. Se l'input supera il test si procede effettuando i test nel sottoalbero sinistro del nodo corrente, mentre, in caso contrario, si procede con il sottoalbero destro. Ogni foglia dell'albero contiene un possibile output (la decisione) che viene raggiunto a seconda dei test superati. Ad esempio, un albero di decisione per decidere se prendere l'ombrello in funzione delle condizioni metereologiche:

![esempio di albero di decisione](albero-di-decisione-esempio.gif "Albero di decisione")

Assumendo che l'input abbia tipo `'a` (nell'esempio della piogga l'input potrebbe essere un record con vari campi relativi ai vari parametri climatici) e che l'output abbia tipo `'b` (nell'esempio, un booleano che indica se prendere l'ombrello o meno), un albero di decisione può essere generalizzato come un albero i cui nodi intermedi contengono un predicato (una funzione) su `'a` che rappresenta il test da effettuare in quel nodo, e le foglie contengono un valore di tipo `'b` (l'output raggiunto). Questa è la definizione del tipo `decision_tree` con costruttori `Nodo` e `Foglia` per le due tipologie di nodo. Si noti le definizioni di tipo con più di un parametro (come in questo caso) richiedono di mettere i parametri tra parentesi).

In [4]:
type ('a,'b) decision_tree = 
  | Nodo of ('a -> bool) * ('a,'b) decision_tree * ('a,'b) decision_tree
  | Foglia of 'b ;;

type ('a, 'b) decision_tree =
    Nodo of ('a -> bool) * ('a, 'b) decision_tree * ('a, 'b) decision_tree
  | Foglia of 'b


**Esercizio 3.2.** Scrivere una funzione `apply_tree` di tipo `('a, 'b) decision_tree -> 'a -> 'b` che, dato un albero di decisione e un possibile input, restituisce l'output corrispondente, eseguendo i test nel modo opportuno

In [6]:
let rec apply_tree dt input =
  match dt with
  | Foglia x -> x
  | Nodo (test,dt1,dt2) -> if test input then apply_tree dt1 input else apply_tree dt2 input;;

val apply_tree : ('a, 'b) decision_tree -> 'a -> 'b = <fun>


**Esercizio 3.3.** Si consideri il seguente tipo che descrive lo stato di salute di un paziente (temperatura corporea e frequenza cardiaca):

In [7]:
type paziente = {temp: float; battiti: int;}

type paziente = { temp : float; battiti : int; }


Si definisca un albero di decisione il cui output è una stringa tra `"Malato"`, `"Alterazione"` e `"Sano"` e i cui test consentano di classificare i pazienti come segue:
* se la temperatura è almeno 38, il paziente è malato
* se la temperatura è almeno 37 e il battito è almeno 80, il paziente è malato
* se la temperatura è almeno 37 e il battito è minore di 80, il paziente ha una alterazione
* se la temperatura è minore di 37 e il battito è almeno 80, il paziente ha una alterazione
* se la temperatura è minore di 37 e il battito è minore di 80, il paziente è sano

**NOTA**: associare il nodo che rappresenta la radice dell'albero alla variabile `radice`

**SUGGERIMENTO**: Procedere in questo modo: (1) pensare all'albero e disegnarlo su carta; (2) definire i nodi uno alla volta partendo dalle foglie e risalendo.

In [13]:
let foglia_sano = Foglia "Sano";;
let foglia_alterazione = Foglia "Alterazione";;
let foglia_malato = Foglia "Malato";;
let nodo3 = Nodo ((fun paz -> paz.battiti >= 80) , foglia_alterazione , foglia_sano);;
let nodo2 = Nodo ((fun paz -> paz.temp >= 37.0) , foglia_alterazione , nodo3);;
let nodo1 = Nodo ((fun paz -> paz.temp >= 37.0 && paz.battiti >= 80) , foglia_malato, nodo2);;
let radice = Nodo ((fun paz -> paz.temp >= 38.0) , foglia_malato, nodo1);;

val foglia_sano : ('a, string) decision_tree = Foglia "Sano"


val foglia_alterazione : ('a, string) decision_tree = Foglia "Alterazione"


val foglia_malato : ('a, string) decision_tree = Foglia "Malato"


val nodo3 : (paziente, string) decision_tree =
  Nodo (<fun>, Foglia "Alterazione", Foglia "Sano")


val nodo2 : (paziente, string) decision_tree =
  Nodo (<fun>, Foglia "Alterazione",
   Nodo (<fun>, Foglia "Alterazione", Foglia "Sano"))


val nodo1 : (paziente, string) decision_tree =
  Nodo (<fun>, Foglia "Malato",
   Nodo (<fun>, Foglia "Alterazione",
    Nodo (<fun>, Foglia "Alterazione", Foglia "Sano")))


val radice : (paziente, string) decision_tree =
  Nodo (<fun>, Foglia "Malato",
   Nodo (<fun>, Foglia "Malato",
    Nodo (<fun>, Foglia "Alterazione",
     Nodo (<fun>, Foglia "Alterazione", Foglia "Sano"))))


Eseguire le seguenti prove per verificare il corretto funzionamento dell'albero

**ATTENZIONE**: in caso qualche prova fallisse l'errore potrebbe essere nella costruzione dell'albero oppure nell'implementazione di `apply_tree`!

In [15]:
(* applicando parzialmente apply_tree ottengo una funzione che fa diagnosi a pazienti*)
let diagnosi = apply_tree radice;; 

let paziente1 = {temp=38.5; battiti=100};;
let paziente2 = {temp=38.5; battiti=70};;
let paziente3 = {temp=37.3; battiti=100};;
let paziente4 = {temp=37.3; battiti=70};;
let paziente5 = {temp=36.8; battiti=100};;
let paziente6 = {temp=36.8; battiti=70};;

diagnosi paziente1;; (* malato *)
diagnosi paziente2;; (* malato *)
diagnosi paziente3;; (* malato *)
diagnosi paziente4;; (* alterato *)
diagnosi paziente5;; (* alterato *)
diagnosi paziente6;; (* sano *)


val diagnosi : paziente -> string = <fun>


val paziente1 : paziente = {temp = 38.5; battiti = 100}


val paziente2 : paziente = {temp = 38.5; battiti = 70}


val paziente3 : paziente = {temp = 37.3; battiti = 100}


val paziente4 : paziente = {temp = 37.3; battiti = 70}


...

- : string = "Malato"


- : string = "Malato"


- : string = "Alterazione"


- : string = "Alterazione"


- : string = "Sano"
